In [1]:
from utils.gmsh_function import *
from src.radiation_algorithm.radiation_algorithm import *
from efield.efield1 import calculate_electric_magnetic_field_at_point
from efield.efield2 import radiation_intensity_distribution_over_sphere_surface
from efield.efield3 import antenna_directivity_pattern

In [2]:
mesh_name = "Monopole_Antenna.msh"
save_mesh_folder = 'data/gmsh_files/'
filename_mesh = 'data/antennas_mesh/Monopole_Antenna.mat'
radiate_strip_msh_gmsh = save_mesh_folder + mesh_name

In [3]:
L_plate = 2.0
W_plate = 2.0
Hight_monopole = 1.0
W_monopole = 0.02

depart_plate_x = -L_plate / 2
depart_plate_y = -W_plate / 2
depart_plate_z = 0

depart_monopole_x = -W_monopole / 2
depart_monopole_y = 0
depart_monopole_z = 0

gmsh.initialize()
gmsh.model.add("Momopole_Antenna")

plate = gmsh.model.occ.addRectangle(-L_plate / 2, -W_plate / 2, 0, L_plate, W_plate)

# Create the monopole
M_P1 = gmsh.model.occ.addPoint(depart_monopole_x, depart_monopole_y, depart_monopole_z)
M_P2 = gmsh.model.occ.addPoint(depart_monopole_x + W_monopole, depart_monopole_y, depart_monopole_z)
M_P3 = gmsh.model.occ.addPoint(depart_monopole_x + W_monopole, depart_monopole_y, depart_monopole_z + Hight_monopole)
M_P4 = gmsh.model.occ.addPoint(depart_monopole_x, depart_monopole_y, depart_monopole_z + Hight_monopole)

M_L1 = gmsh.model.occ.addLine(M_P1, M_P2)
M_L2 = gmsh.model.occ.addLine(M_P2, M_P3)
M_L3 = gmsh.model.occ.addLine(M_P3, M_P4)
M_L4 = gmsh.model.occ.addLine(M_P4, M_P1)

Loop_monopole = gmsh.model.occ.addCurveLoop([M_L1, M_L2, M_L3, M_L4])
Surface_monopole = gmsh.model.occ.addPlaneSurface([Loop_monopole])

# Fusionner les deux surfaces
monopole_antenna, _ = gmsh.model.occ.fuse([(2, plate)], [(2, Surface_monopole)])

gmsh.model.occ.synchronize()

apply_mesh_size(L_plate/10)
gmsh.model.mesh.generate(2)
# run()

write(save_mesh_folder, mesh_name)

gmsh.finalize()

In [4]:
extract_msh_to_mat(radiate_strip_msh_gmsh, filename_mesh)

In [ ]:
# Mesure du temps de début
start_time = time.time()

feed_point = np.array([[0, 0, 0]])
frequency = 75e6

radiation_algorithm(filename_mesh, frequency, feed_point, monopole=True)

elapsed_time = time.time() - start_time
print(f"Temps écoulé pour le traitement de l'antenne étudiée : {elapsed_time:.6f} secondes")
print('\n')


Nombre de triangles = 578
Suppression de 3 jonctions en T.

Nombre d'elements de maillage (edges) = 842

index_feeding_edges = [798 797]
voltage[index_feeding_edges] =  [0.02+0.j 0.02+0.j]

La valeur de l'impédance d'entrée de l'antenne Monopole_Antenna = 28.1012248 + 21.7051625i Ohm
Gap current of Monopole_Antenna = (0.022288533250239804-0.017215485774086874j)
Gap voltage of Monopole_Antenna = (1+0j)
La valeur de feed_power  = 0.011144266625119902


Surface current density of Monopole_Antenna: min = 0.0004505 [A/m], max = 1.4234038 [A/m]



Temps écoulé pour le traitement de l'antenne étudiée : 1.175476 secondes




# Maillage dense pour une sphère utilisée dans les calculs de rayonnement
filename_sphere_dense = '../../data/sphere_mesh/sphere_dense.mat'

mesh2_mat = 'data/antennas_mesh2/array_linear_1_mesh2.mat'
current_mat = 'data/antennas_current/array_linear_1_current.mat'
gain_power_mat = 'data/antennas_gain_power/array_linear_1_gain_power.mat'

observationPoint = np.array([100, 0, 0])

calculate_electric_magnetic_field_at_point(mesh2_mat, current_mat, observationPoint, radiation=True)

print("Calculating radiation intensity distribution over sphere surface...")
radiation_intensity_distribution_over_sphere_surface(mesh2_mat, current_mat, filename_sphere_dense, radiation=True)

# Étape 3 : Génération du diagramme de directivité
print("Generating antenna directivity pattern...")
antenna_directivity_pattern(mesh2_mat, current_mat, gain_power_mat, radiation=True)